# 1. ChatGPT API 를 사용해 보완점 & 반대 의견 논문 제공

In [ ]:
!pip install OpenAI

In [ ]:
import os
from openai import OpenAI
#import pandas as pd
import openai

In [ ]:
# user essay
user_essay = '''
The Acceleration of Global Warming Due to Carbon Emissions
Global warming is one of the most pressing challenges of our time, driven largely by carbon emissions from human activities. The burning of fossil fuels, such as coal, oil, and natural gas, has significantly increased greenhouse gas concentrations in the atmosphere, intensifying the natural greenhouse effect. This has caused global temperatures to rise, leading to melting ice caps, rising sea levels, and more frequent extreme weather events like hurricanes and heatwaves. Ecosystems and human societies are already feeling the impacts of these changes.
Energy production, transportation, industrial processes, and agriculture are the primary sources of carbon emissions. Fossil fuel reliance for electricity, heating, and transportation is a major contributor, while deforestation and livestock farming also exacerbate the issue. Addressing these emissions requires a shift to renewable energy sources, improved energy efficiency, and widespread adoption of sustainable practices. On a global scale, agreements like the Paris Climate Accord emphasize the importance of international collaboration to curb emissions.
While the challenges are immense, solutions are within reach. Transitioning to renewable energy, reducing consumption, and supporting green technologies can mitigate the worst effects of global warming. The climate crisis demands urgent and collective action, as the future of the planet and its inhabitants depends on the choices made today.
'''

In [ ]:
# instruction to provide additional viewpoints
system_prompt_opposing_viewpoints = '''You are a critical writer tasked with analyzing the user's essay.
The user is seeking opposing viewpoints to challenge the arguments presented.
Review the essay carefully and construct well-reasoned opposing viewpoints according to the following rules:

[Rules]
- Construct a minimum of three well-reasoned opposing viewpoints.
- Use the following response format: 1. ...: Explanation providing logical reasoning and context.
- The Main Point should be concise, expressed as a short phrase, sentence, or words.
- Number each response sequentially.
- Begin each response with the main point, followed by a clear and detailed explanation.
- Start directly with the main point. Avoid using introductory phrases such as "Opposing Viewpoint", "Main Point" or any labels before the main point.
- Do not include words like "Opposing Viewpoint", "Main Point"'''

In [ ]:
# instruction to provide constructive feedback
system_prompt_constructive_feedback = '''You are a critical reviewer.
Read and analyze the user's essay and provide constructive feedback by identifying areas for improvement or enhancement according to the following rules:

[Rules]
- Respond only with specific suggestions for strengthening the essay.
- Start directly with the feedback and do not include introductory or concluding remarks.
- Use the following response format: 1. ... : Explanation of the suggestion.'''

In [ ]:
from openai import OpenAI
client = OpenAI()

# Generate response for opposing viewpoints
response_opposing = client.chat.completions.create(
  model="gpt-3.5-turbo-0125",
  messages=[
    {
      "role": "system",
      "content": system_prompt_opposing_viewpoints
    },
    {
      "role": "user",
      "content": user_essay
    }
  ],
  temperature=0.2,   # More creative response
  presence_penalty=0.3    # Encourage exploration of new ideas
)


generated_opposing_viewpoints = response_opposing.choices[0].message.content
print(generated_opposing_viewpoints)

1. Economic Impact of Transitioning to Renewable Energy: Transitioning to renewable energy sources may have significant economic implications, including job losses in the fossil fuel industry and increased costs for consumers.
2. Technological Limitations of Green Technologies: The current green technologies may not be advanced enough to fully replace fossil fuels in all sectors, leading to potential gaps in energy supply and reliability.
3. Social and Political Barriers to International Collaboration: Achieving global agreements like the Paris Climate Accord may be hindered by political conflicts, differing national interests, and social resistance to change, making it challenging to implement effective emissions reduction strategies on a global scale.


In [ ]:
# Generate response for constructive feedback
response_feedback = client.chat.completions.create(
  model="gpt-3.5-turbo-0125",
  messages=[
    {
      "role": "system",
      "content": system_prompt_constructive_feedback
    },
    {
      "role": "user",
      "content": user_essay
    }
  ],
  temperature=0
)

generated_constructive_feedback = response_feedback.choices[0].message.content
def format_feedback(feedback_text):
    formatted_text = feedback_text.replace('\n\n', '\n')
    return formatted_text

#print(generated_constructive_feedback)
print(format_feedback(generated_constructive_feedback))

1. Provide specific examples and data: To strengthen the essay, consider incorporating specific examples and data to support the claims made. For instance, you could include statistics on the amount of carbon emissions produced by different sectors or the impact of global warming on specific regions or species. This will enhance the credibility and persuasiveness of your argument.
2. Include counterarguments: To make the essay more comprehensive, address potential counterarguments or skepticism towards the issue of global warming and carbon emissions. By acknowledging and refuting opposing viewpoints, you can demonstrate a deeper understanding of the topic and strengthen your position.
3. Expand on the consequences: While you briefly mention the impacts of global warming such as melting ice caps and extreme weather events, consider expanding on these consequences. Describe in more detail how these effects are affecting ecosystems, communities, and economies around the world. Providing 

# 2. Open Source LLM을 사용해 보완점 & 반대 의견 논문 제공

### Model1: meta-llama__Meta-Llama-3.1-70B-Instruct-details
### Model2: dfurman/CalmeRys-78B-Orpo-v0.1

[open-llm-leaderboard](https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard)

## Llma-3.1-70B
### [Model Link](https://huggingface.co/meta-llama/Llama-3.1-70B-Instruct)

In [ ]:
!pip install transformers huggingface_hub

In [ ]:
import transformers
import torch

model_id = "meta-llama/Meta-Llama-3.1-70B-Instruct"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]

outputs = pipeline(
    messages,
    max_new_tokens=256,
)
print(outputs[0]["generated_text"][-1])

## CalmeRys-78B0-Orpo-v0.1

###[Model Link](https://huggingface.co/dfurman/CalmeRys-78B-Orpo-v0.1)

In [ ]:
!pip -q install -qU transformers accelerate bitsandbytes
!huggingface-cli download dfurman/CalmeRys-78B-Orpo-v0.1

In [ ]:
from transformers import AutoTokenizer, BitsAndBytesConfig
import transformers
import torch

if torch.cuda.get_device_capability()[0] >= 8:
    !pip install -qqq flash-attn
    attn_implementation = "flash_attention_2"
    torch_dtype = torch.bfloat16
else:
    attn_implementation = "eager"
    torch_dtype = torch.float16

# quantize if necessary
bnb_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_compute_dtype=torch_dtype,
   bnb_4bit_use_double_quant=True,
)

model = "dfurman/CalmeRys-78B-Orpo-v0.1"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    model_kwargs={
        "torch_dtype": torch_dtype,
        "quantization_config": bnb_config,
        "device_map": "auto",
        "attn_implementation": attn_implementation,
    }
)

In [ ]:
question = "Is the number 9.11 larger than 9.9?"

messages = [
    {"role": "system", "content": "You are a helpful assistant that thinks step by step."},
    {"role": "user", "content": question},
]
prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
print("***Prompt:\n", prompt)

outputs = pipeline(
    prompt, max_new_tokens=1000, do_sample=True, temperature=0.7, top_k=50, top_p=0.95
)
print("***Generation:")
print(outputs[0]["generated_text"][len(prompt) :])

NameError: name 'tokenizer' is not defined